<a href="https://colab.research.google.com/github/aimlbits16/Group_16/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2.1	Problem definition:

Predict denials of denial claims from history data

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

2.2	History Data Set 
Data in excel contains three tabs

	Tab-1 contains all attributes of the claims processed except Label   
	Tab-2 contains only Target variable (Label) of each row of Tab-1 data with one common attribute  
	Tab-3 contains all attribute list 

2.3 Data Pre-processing 

1.Read Tab1- data set and Tab-2 Data set and create two panda data frames

2.Merge two data frames into one to place Label properly against each row.

3.Attributes of Tab-1 and Tab-2 are shown below

In [0]:
# Library Imports

import sys
print('python:{}'.format(sys.version))
import scipy as sp
print('scipy:{}'.format(sp.__version__))
import numpy as np
print('numpy:{}'.format(np.__version__))
import pandas as pd
print('pandas:{}'.format(pd.__version__))

python:3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]
scipy:1.4.1
numpy:1.16.5
pandas:0.25.1


In [0]:
# Data Reads - All required data is in data folders
InputFolder = '/content/gdrive/My Drive/capstone-project/input_data'
OutFolder = '/content/gdrive/My Drive/capstone-project/output_data'
inFile = InputFolder + '/ZCUPClaimDNA-History-Data-v2.xlsx'
df_history_dataset = pd.read_excel(inFile, 
                                   sheet_name='HistoryData',
                                   header=0,
                                   converters={'srlno':np.int64, 
                                               'Claim.Number':str,
                                               'Claim.Line.Number':str,
                                               'Member.ID':str,
                                               'Provider.ID':str,
                                               'Revenue.Code':str,
                                               'Service.Code':str,
                                               'Place.Of.Service.Code':str,
                                               'Procedure.Code':str,
                                               'Diagnosis.Code':str,
                                               'Claim.Charge.Amount':np.float64,
                                               'Denial.Reason.Code':str,
                                               'In.Out.Of.Network':str,
                                               'Subscriber.Payment.Amount':np.float64,
                                               'Provider.Payment.Amount':np.float64,
                                               'Claim.Type':str,
                                               'Claim.Subscriber.Type':str,
                                               'Claim.Current.Status':str,
                                               'Network.ID':str,
                                               'Agreement.ID':str
                                              })
df_labels_dataset = pd.read_excel(inFile, 
                                  sheet_name='Accept-Denial-Label', 
                                  header=0,
                                  converters={'srlno':np.int64, 
                                              'Label':np.int64
                                             })

df_data = pd.merge(df_history_dataset, df_labels_dataset, on='Srlno')

print(df_history_dataset.head())
print(df_labels_dataset.head())

print(df_history_dataset.describe())
print(df_labels_dataset.describe())

print(df_data.head())
print(df_data.describe())

   Srlno Claim.Number Claim.Line.Number Member.ID   Provider.ID Revenue.Code  \
0      1     28849991                 1   9583423  FA0001411002          128   
1      2     81404290                 1  25210465  FA0001411002           24   
2      3     81404290                 2  25210465  FA0001411002          128   
3      4     81404290                 3  25210465  FA0001411002          250   
4      5     81404290                 4  25210465  FA0001411002          258   

  Service.Code Place.Of.Service.Code Procedure.Code Diagnosis.Code  \
0         RR8I                                               V5789   
1         AL7I                                A0205          V5789   
2         RR8I                                               V5789   
3         AL7I                                               V5789   
4         AL7I                                               V5789   

   Claim.Charge.Amount Denial.Reason.Code In.Out.Of.Network  \
0         19356.960000             

2.4 Find Shape and data type 

In [0]:
print(df_data.shape)
dataTypeSeries = df_data.dtypes
print(dataTypeSeries)

(472559, 21)
Srlno                          int64
Claim.Number                  object
Claim.Line.Number             object
Member.ID                     object
Provider.ID                   object
Revenue.Code                  object
Service.Code                  object
Place.Of.Service.Code         object
Procedure.Code                object
Diagnosis.Code                object
Claim.Charge.Amount          float64
Denial.Reason.Code            object
In.Out.Of.Network             object
Subscriber.Payment.Amount    float64
Provider.Payment.Amount      float64
Claim.Type                    object
Claim.Subscriber.Type         object
Claim.Current.Status          object
Network.ID                    object
Agreement.ID                  object
Label                          int64
dtype: object


2.5	Target Value (Label) setting and cleaning

2.5.1    We will consider following valid Denial.Reason.Code F13, J8G, JO5, JB8, JE1, JC9, JF1, JF9, JG1, JPA and JES
The goal is to develop predictive model to predict claims that would be denied with the above Denial.Reason.Code only. Any row with non-valid Denial.Reason.Code will be considered as garbage that need to be deleted

In [0]:
print(df_data.groupby('Denial.Reason.Code')['Denial.Reason.Code'].count())

validReasonCodes = ['F13', 'J8G', 'JO5', 'JB8', 'JE1', 'JC9', 'JF1', 'JF9', 'JG1', 'JPA', 'JES',' ']
df_data=df_data[(df_data['Denial.Reason.Code'].isin(validReasonCodes))]

print(df_data.groupby('Denial.Reason.Code')['Denial.Reason.Code'].count())

Denial.Reason.Code
       326030
104        44
17          3
18          1
19        242
        ...  
TF1      1031
TR2        22
UM0         2
UM1        28
UM2         7
Name: Denial.Reason.Code, Length: 259, dtype: int64
Denial.Reason.Code
       326030
F13      1068
J8G       382
JB8       128
JC9        49
JE1        33
JES        50
JF1         4
JF9        98
JG1        10
JO5       105
JPA        44
Name: Denial.Reason.Code, dtype: int64


2.5.2   Check distribution of Label in terms of percentage 0/1

In [0]:
distLabels = (df_data['Label'].value_counts()/ df_data['Label'].count())*100
print('Distribution:\n{}'.format(distLabels))

Distribution:
0    99.399087
1     0.600913
Name: Label, dtype: float64


2.5.3   Find number of  distinct valid Denial.Reason.Code

In [0]:
print(df_data_1['Denial.Reason.Code'].unique())

['JB8' 'J8G' 'F13' 'JF1' 'JE1' 'JES' 'JO5' 'JG1' 'JPA' 'JC9' 'JF9']


2.6	Handling Predictor Variables
Predictors variables ( All input variables except Target variable)  that are used to predict the 'Target Variable'.


2.6.1  Identify

•	continuous numeric value
•	tag/code value
•	Other columns ID type.

2.6.2  Find

In [0]:
print(df_data['Revenue.Code'].unique())
print(df_data['Service.Code'].unique())
print(df_data['Diagnosis.Code'].unique())

['128' '24' '250' '258' '271' '300' '301' '305' '306' '307' '309' '420'
 '424' '430' '434' '440' '444' '636' '351' '324' '352' '410' '730' '361'
 '302' '320' '350' '390' '274' '370' '921' '450' '915' '912' '126' '124'
 '914' '771' '483' '913' '740' '121' '402' '905' '341' '343' '482' '762'
 '949' '431' '421' '761' '359' '333' '943' '441' '433' '272' '490' '710'
 '278' '259' '750' '312' '510' '360' '637' '329' '206' '200' '460' '481'
 '270' '275' '480' '276' '611' '310' '314' '261' '369' '920' '940' '941'
 '911' '919' '917' '111' '260' '403' '279' '391' '918' '801' '999' '311'
 '610' '612' '401' '323' '344' '731' '981' '412' '812' '127' '322' '819'
 '404' '829' '131' '633' '615' '335' '614' ' ' '922' '822' '400' '732'
 '25' '929' '948' '942' '622' '616' '342' '129' '635' '634' '118' '331'
 '443' '255' '136' '134' '254' '117' '249' '100' '122' '171' '720' '210'
 '723' '821' '722' '471' '112' '172' '900' '916' '123' '120' '618' '174'
 '205' '790' 'XXXX' '173' '831' '901' '413' '832' '811'

2.6.3  Handling Categorical Values

In [0]:
#2.6.3.1 Clubbing of Category Value (1 ,2,3 – High , Medium , Low possibility group respectively)

# Revenue Code
High_Revenue_Code = ["0810", "0811", "0124", "0261", "0260", "0331", "0914", "0174", " ", "0,634", "0173", "0682", "0126", "0128", "0172", "0127", "0200", "0821", "0134", "0302", "0740", "0249", "0270", "0771", "0361", "0310", "0434"]
Medium_Revenue_Code= ["0210", "0275", "0122", "0610", "0206", "0258", "0444", "0278", "0117", "0259", "0121", "0112", "0276", "0731", "0750", "0460", "0360", "0271", "0730", "0307", "0424", "0370", "0250", "0410", "0940", "0305", "0350", "0921", "0480", "0111", "0351", "0920", "0636", "0309", "0300", "0390", "0301", "0450", "0440", "0306", "0311", "0324", "0710", "0341", "0272", "0343", "0942", "0171", "0482", "0420", "0490", "0483", "0312", "0481", "0352", "0510", "0402", "0320", "0720", "0762", "0761", "0430", "0403"]

df_data.loc[df_data['Revenue.Code'].isin(High_Revenue_Code)]=1
df_data.loc[df_data['Revenue.Code'].isin(Medium_Revenue_Code)]=2
df_data.loc[~df_data['Revenue.Code'].isin(High_Revenue_Code).isin(Medium_Revenue_Code)]=3


# Service Code
d = {
'1' : {"UNI", "RL8Y", "RM8W", "DU7O", "IH7O", "RM8I", "RI7Y", "SB7W", "GB7A", "NI7I", "OVV", "RM9I", "RR8I", "IMY", "RT8I", "RY8W", "RY7I", "IC7I", "SU7A", "ER7I"},
'2' : {"TO7W", "TE7O", "CC7I", "HV7I", "MR7O", "IN7I", "AL7I", "DI7O", "CT8Y", "RB7W", "RV8W", "CT7I", "RB8I", "RK8Y", "RB7I", "OE7I", "RY8I", "OP7O", "LA7O", "NS7I", "CL7O", "MR7Y", "ER7O", "CT7Y", "CT7O", "RA7O", "MD7O", "TH7O"},
'3' : {"Rest"}
}

newdict = {i:k for k, v in d.items() for i in v}
df_data['Service.Code'].str.findall('|'.join(newdict.keys())).str[0].map(newdict)
df_data['Service.Code']=df_data['Service.Code'].str.findall('|'.join(newdict.keys())).str[0].map(newdict)

# Diagnosis Code
d = {
'1' : {"2929", "29690", "V202", "29663", "29622", "9962", "67524", "5193", "V045", "29530", "7999", "430", "29664", "5711", "79095", "30411", "29189", "6162", "29570", "29630", "29644", "5289", "20240", "99741", "37943", "9352", "V08", "V1255", "V5041", "80225", "6108", "57480", "5070", "V6284", "52460", "8024", "99681", "V3101"},
'2' : {"75249", "71100", "29660", "78722", "2273", "6868", "64844", "73671", "37730", "61804", "6146", "V5831", "2989", "7503", "72992", "29181", "2452", "80841", "8250", "66111", "9950", "29640", "9220", "99563", "7222", "29650", "65811", "29632", "52403", "99676", "27541", "4168", "2102", "2182", "0090", "34691", "5738", "68100", "2776", "1809", "5362", "29633", "29680", "61801", "34290", "72990", "64981", "2912", "6170", "1746", "6183", "78499", "52489", "53551", "42823", "30501", "3682", "V3000", "7226", "64303", "24200", "8832", "57470", "9726", "29689", "5409", "8248", "73730", "2181", "57450", "85011", "6121", "64893", "6253", "99883", "96501", "193", "53190", "6268", "42841", "226", "57400", "4280", "1960", "69589", "2777", "2180", "1743", "1742", "179", "6110", "6272", "60001", "53240", "V552", "6827", "5559", "6185", "8600", "81305", "07054", "0479", "45340", "6824", "5551", "38611", "5941", "2271", "72888", "80703", "6227", "6823", "6184", "4377", "62212", "72210", "2891", "5533", "5722", "8246", "V510", "6111", "2572", "2720", "5789", "55321", "3688", "2189", "2410", "1541", "5856", "65231", "6262", "5609", "5641", "7455", "1550", "2875", "220", "78720", "30300", "81209", "566", "99812", "56032", "4321", "1745", "78630", "5119", "5400", "34590", "57511", "30400", "4940", "5770", "25201", "75612", "78057", "6181", "61179", "1749", "27651", "73819", "78799", "00845", "56400", "99832", "5712", "4589", "42741", "78701", "4659", "57420", "30500", "7804", "30391", "5693", "1744", "6100", "41401", "55092", "5589", "61172", "99674", "1741", "61189", "49392", "82525", "9351", "V5331", "2910", "78901", "5303", "80126", "6271", "2141", "7842", "59970", "25002", "78791", "1748", "8500", "78900", "217", "1985", "79389", "42731", "78906", "99673", "7384", "78009", "650", "6210", "51881", "81342", "78659", "72402", "56210", "2851", "49322", "486", "55090", "2920", "56211", "V700", "57410", "4359", "5921", "03849", "78903", "53550", "7840", "2330", "64403", "33819", "6826", "78907", "V5811", "6202", "78652", "431", "V6759", "78079", "72252", "7802", "2809", "V7651", "7295", "78650", "65421", "0389", "43491", "78909", "2859", "V7611", "V7612", "3669", "V5789", "V571"},
'3' : {"Rest"}
}

newdict = {i:k for k, v in d.items() for i in v}
df_data['Diagnosis.Code'].str.findall('|'.join(newdict.keys())).str[0].map(newdict)
df_data['Diagnosis.Code']=df_data['Diagnosis.Code'].str.findall('|'.join(newdict.keys())).str[0].map(newdict)

print(df_data['Revenue.Code'].unique())
print(df_data['Service.Code'].unique())
print(df_data['Diagnosis.Code'].unique())

NameError: name 'df_data' is not defined

2.6.4          Find column wise missing value in percentage  

In [0]:
percent_missing = df_data.isnull().sum() *100 / len(df_data)
print(percent_missing)

Srlno                         0.000000
Claim.Number                 53.055631
Claim.Line.Number             0.000000
Member.ID                     0.000000
Provider.ID                   0.000000
Revenue.Code                  0.000000
Service.Code                  0.000000
Place.Of.Service.Code         0.000000
Procedure.Code                0.000000
Diagnosis.Code                0.000000
Claim.Charge.Amount           0.000000
Denial.Reason.Code            0.000000
In.Out.Of.Network             0.000000
Subscriber.Payment.Amount     0.000000
Provider.Payment.Amount       0.000000
Claim.Type                    0.000000
Claim.Subscriber.Type         0.000000
Claim.Current.Status          0.000000
Network.ID                    0.000000
Agreement.ID                  0.000000
Label                         0.000000
dtype: float64


2.6.5	       Find Duplicate rows and drop those

In [0]:
df_data=df_data.drop_duplicates(subset=None, keep = 'first', inplace=False)

2.6.6	       Drop all useless columns – not required for Machine learning

2.6.7	       Check distribution of Label in terms of percentage 0/1

In [0]:
distLabels = (df_data['Label'].value_counts()/ df_data['Label'].count())*100
print(distLabels)

0    99.399087
1     0.600913
Name: Label, dtype: float64


2.6.8	       Write processed and clean data set  to a csv file

In [0]:
df_final.to_csv(OutFolder+"\\CleanData.csv",index=False)    